## Setup file path manager

In [1]:
from data_preparation import filepaths

path_manager = filepaths.PathManager("./data")


## Download audio files and metadata from xeno-canto

In [2]:
from data_preparation import downloader

downloader = downloader.XenoCantoDownloader(path_manager)

downloader.create_datasets("nips4b_species_list.csv", min_quality="A", sound_types=["song", "call"],    maximum_number_of_background_species=1)

Multiple species found for noise sample
Skipping class noise sample


Download label file for Chloris chloris...: 100%|██████████| 4/4 [00:07<00:00,  2.01s/it]

Multiple species found for Cicadatra atra
Skipping class Cicadatra atra
Multiple species found for Cicada orni
Skipping class Cicada orni


Download label file for Luscinia megarhynchos...: 100%|██████████| 4/4 [00:07<00:00,  1.99s/it]

Multiple species found for Lyristes plebejus
Skipping class Lyristes plebejus


...

Download label file for Prunella modularis...: 100%|██████████| 4/4 [00:07<00:00,  1.97s/it]

Multiple species found for Pteronemobius heydenii
Skipping class Pteronemobius heydenii


Download label file for Sylvia undata...: 100%|██████████| 1/1 [00:00<00:00, 19.61it/s]


Multiple species found for Tettigettula pygmea
Skipping class Tettigettula pygmea
Multiple species found for Tibicina tomentosa
Skipping class Tibicina tomentosa


Download label file for Passer domesticus...: 100%|██████████| 4/4 [02:39<00:00, 39.80s/it]4s/it]
Download label file for Phylloscopus collybita...: 100%|██████████| 9/9 [02:24<00:00, 16.05s/it]
Download test set:  99%|█████████▊| 922/935 [07:53<00:13,  1.03s/it].21it/s]:00,  2.48s/it]6s/it]

## Create spectrograms from audio files

In [3]:
from data_preparation import spectrograms

spectrogram_creator = spectrograms.SpectrogramCreator(
    1000, path_manager)

spectrogram_creator.create_spectrograms_for_splits(datasets=["train", "val", "test"])

Download test set: 100%|██████████| 935/935 [12:46<00:00,  1.22it/s]
Download label file for Troglodytes troglodytes...: 100%|██████████| 7/7 [1:06:01<00:00, 565.92s/it]
Create spectrograms for training set: 100%|██████████| 3074/3074 [3:33:24<00:00,  4.68s/it]   

NameError: No matching labels found for file with id 100542

## Create datasets and dataloaders

In [4]:
from training import dataset
from torch.utils.data import DataLoader

train_set = dataset.XenoCantoSpectrograms(
    path_manager, chunk_length=1000, split="train")
train_loader = DataLoader(
    train_set, batch_size=2, shuffle=True, num_workers=0)


val_set = dataset.XenoCantoSpectrograms(
    path_manager, chunk_length=1000, split="val")
val_loader = DataLoader(
    val_set, batch_size=2, shuffle=True, num_workers=0)

test_set = dataset.XenoCantoSpectrograms(
    path_manager, chunk_length=1000, split="test")
test_loader = DataLoader(
    test_set, batch_size=2, shuffle=True, num_workers=0)

## Train model

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
from torchvision import models

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('----------')

        # Training Phase
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for images_train, labels_train in train_loader:
            # TODO: Labels muessen hier noch richtig als Tensor übergeben werden
            labels_train = torch.zeros(images_train.shape[0], dtype=torch.long)
            images_train = images_train.to(device)
            labels_train = labels_train.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                images_out = model(images_train)
                _, preds = torch.max(images_out, 1)
                loss = criterion(images_out, labels_train)

                loss.backward()
                optimizer.step()

            running_loss += loss.item() * images_train.size(0)
            running_corrects += torch.sum(preds == labels_train.data)

            scheduler.step()

            epoch_loss = running_loss / len(train_set)
            epoch_acc = running_corrects.double() / len(train_set)

           # print('{} Loss: {:.4f} Acc: {:.4f}'.format(
           #     'train', epoch_loss, epoch_acc))

            # Evaluate Model
            model.eval()
            running_loss = 0.0
            running_corrects = 0

            for images_val, labels_val in val_loader:
                # TODO: Labels muessen hier noch richtig als Tensor übergeben werden
                labels_val = torch.zeros(images_val.shape[0], dtype=torch.long)
                images_val = images_val.to(device)
                labels_val = labels_val.to(device)

                optimizer.zero_grad()

                outputs = model(images_val)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels_val)

                running_loss += loss.item() * images_val.size(0)
                running_corrects += torch.sum(preds == labels_val.data)

            epoch_loss = running_loss / len(val_set)
            epoch_acc = running_corrects.double() / len(val_set)

           # print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            #    'val', epoch_loss, epoch_acc))

            if epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

resnet_model = models.resnet18()
resnet_model.to(device)

my_criterion = nn.CrossEntropyLoss()
my_optimizer = optim.SGD(resnet_model.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(my_optimizer, step_size=7, gamma=0.1)

my_model = train_model(resnet_model, my_criterion, my_optimizer, exp_lr_scheduler,
                       num_epochs=25)

cuda:0
Epoch 0/24
----------


FileNotFoundError: [Errno 2] No such file or directory: './data\\train\\spectrograms_1000\\433015-168.png'